# Inspect and summarize SLURM benchmarks

## Overview

## Load, merge and compare benchmark summary `csv`s

We need a function to compare our SLURM summary output files and highlight differences in a table.

In [1]:
library(dplyr)
library(tidyr)

read_and_summarize_differences <- function(directory) {
  # List all CSV files in the directory
  files <- list.files(directory, pattern = "\\.csv$", full.names = TRUE)
  files <- files[!grepl("1000000", files)]
  
  # Read each file and store in a list
  data_list <- lapply(files, function(file) {
    data <- read.csv(file)
    # Add a column to identify the source file
    data$Source <- basename(file)
    return(data)
  })
  
  # Combine all data frames into one
  combined_data <- bind_rows(data_list)
  
  # Spread the data to wide format for comparison
  wide_data <- spread(combined_data, key = Parameter, value = Value)
  
  # Identify rows with at least one difference in values
  differing_rows <- wide_data %>%
    summarise(across(-Source, ~n_distinct(.) > 1)) %>%
    select(which(colSums(.) > 0)) %>%
    names()
  
  # Filter the combined data to include only differing parameters
  summary_data <- combined_data %>%
    filter(Parameter %in% differing_rows) %>%
    select(Parameter, Source, Value)
  
  # Pivot wider for a clearer summary table, if necessary
  summary_table <- summary_data %>%
    pivot_wider(names_from = Source, values_from = Value)
  
  return(summary_table)
}


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
getwd()

[1] "/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/scripts"

In [3]:
directory <- "/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/output_chr1_a2_caud/"
summary_table <- read_and_summarize_differences(directory)
summary_table

Parameter,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1008982-1028981-dynamic-1corestotal-allcorepera-20240416-154932-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1028982-1048981-dynamic-1corestotal-allcorepera-20240415-140831-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1048982-1068981-dynamic-1corestotal-allcorepera-20240415-141231-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1068982-1088981-dynamic-1corestotal-allcorepera-20240415-141631-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1088982-1108981-dynamic-1corestotal-allcorepera-20240415-142031-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-108982-128981-dynamic-1corestotal-allcorepera-20240416-154224-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1108982-1128981-dynamic-1corestotal-allcorepera-20240415-142431-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1128982-1148981-dynamic-1corestotal-allcorepera-20240415-142832-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1148982-1168981-dynamic-1corestotal-allcorepera-20240415-143232-summary.csv,⋯,libd_chr1-chr1_EA-libd_chr1-chr1_EA-868982-888981-dynamic-1corestotal-allcorepera-20240419-170914-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-888982-908981-dynamic-1corestotal-allcorepera-20240419-171216-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-88982-108981-dynamic-1corestotal-allcorepera-20240419-151129-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-8982-28981-dynamic-1corestotal-allcorepera-20240419-145925-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-8982-28981-dynamic-1corestotal-allcorepera-20240421-170133-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-908982-928981-dynamic-1corestotal-allcorepera-20240419-171517-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-928982-948981-dynamic-1corestotal-allcorepera-20240419-171818-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-948982-968981-dynamic-1corestotal-allcorepera-20240419-172119-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-968982-988981-dynamic-1corestotal-allcorepera-20240419-172420-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-988982-1008981-dynamic-1corestotal-allcorepera-20240419-172721-summary.csv
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
outdir,./output_chr1_a2_caud/,./output_chr1_a2/,./output_chr1_a2/,./output_chr1_a2/,./output_chr1_a2/,./output_chr1_a2_caud/,./output_chr1_a2/,./output_chr1_a2/,./output_chr1_a2/,⋯,./output_chr1_a2_caud/,./output_chr1_a2_caud/,./output_chr1_a2_caud/,./output_chr1_a2_caud/,./output_chr1_a2_caud/,./output_chr1_a2_caud/,./output_chr1_a2_caud/,./output_chr1_a2_caud/,./output_chr1_a2_caud/,./output_chr1_a2_caud/
chunk2,20000,20000,20000,20000,20000,20000,20000,20000,20000,⋯,20000,20000,20000,20000,20000,20000,20000,20000,20000,20000
methylation_data_path,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,⋯,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/

## Compute core-hours per test

In [4]:
windows <- c(10000)
n_windows <- length(windows)
alphas <- seq(0.5) # left out 0 on first run
n_alphas <- length(alphas)
#chunks <- 10000
#n_tests <- n_windows * n_alphas * chunks
#n_tests

In [5]:
alphas

[1] 1

In [6]:
colnames(summary_table)

[1] "Parameter"                                                                                                        
  [2] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1008982-1028981-dynamic-1corestotal-allcorepera-20240416-154932-summary.csv"  
  [3] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1028982-1048981-dynamic-1corestotal-allcorepera-20240415-140831-summary.csv"  
  [4] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1048982-1068981-dynamic-1corestotal-allcorepera-20240415-141231-summary.csv"  
  [5] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1068982-1088981-dynamic-1corestotal-allcorepera-20240415-141631-summary.csv"  
  [6] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1088982-1108981-dynamic-1corestotal-allcorepera-20240415-142031-summary.csv"  
  [7] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-108982-128981-dynamic-1corestotal-allcorepera-20240416-154224-summary.csv"    
  [8] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1108982-1128981-dynamic-1corestotal-allcorepera-20240415-142431-summary.csv"  
  [9] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1128982-1148981-dynamic-1corestotal-allcorepera-20240415-142832-summary.csv"  
 [10] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1148982-1168981-dynamic-1corestotal-allcorepera-20240415-143232-summary.csv"  
 [11] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1168982-1188981-dynamic-1corestotal-allcorepera-20240415-143632-summary.csv"  
 [12] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1188982-1208981-dynamic-1corestotal-allcorepera-20240415-144032-summary.csv"  
 [13] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1208982-1228981-dynamic-1corestotal-allcorepera-20240415-144432-summary.csv"  
 [14] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1228982-1248981-dynamic-1corestotal-allcorepera-20240415-144833-summary.csv"  
 [15] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1248982-1268981-dynamic-1corestotal-allcorepera-20240415-145233-summary.csv"  
 [16] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1248982-1268981-dynamic-1corestotal-allcorepera-20240416-155033-summary.csv"  
 [17] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1248982-1268981-dynamic-1corestotal-allcorepera-20240419-112750-summary.csv"  
 [18] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1248982-1268981-dynamic-1corestotal-allcorepera-20240421-164124-summary.csv"  
 [19] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1268982-1288981-dynamic-1corestotal-allcorepera-20240415-145633-summary.csv"  
 [20] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1268982-1288981-dynamic-1corestotal-allcorepera-20240416-155134-summary.csv"  
 [21] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1268982-1288981-dynamic-1corestotal-allcorepera-20240419-113051-summary.csv"  
 [22] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1268982-1288981-dynamic-1corestotal-allcorepera-20240421-164525-summary.csv"  
 [23] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1288982-1308981-dynamic-1corestotal-allcorepera-20240415-150033-summary.csv"  
 [24] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-128982-148981-dynamic-1corestotal-allcorepera-20240415-110821-summary.csv"    
 [25] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1308982-1328981-dynamic-1corestotal-allcorepera-20240415-150433-summary.csv"  
 [26] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1328982-1348981-dynamic-1corestotal-allcorepera-20240415-150833-summary.csv"  
 [27] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1348982-1368981-dynamic-1corestotal-allcorepera-20240415-151234-summary.csv"  
 [28] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1368982-1388981-dynamic-1corestotal-allcorepera-20240415-151634-summary.csv"  
 [29] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1388982-1408981-dynamic-1corestotal-allcorepera-20240416-155235-summary.csv"  
 [30] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1388982-1408981-dynamic-1corestotal-allcorepera-20240419-113352-summary.csv"  
 [31] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1388982-1408981-dynamic-1corestotal-allcorepera-20240421-164926-summary.csv"  
 [32] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1408982-1428981-dynamic-1corestotal-allcorepera-20240416-155336-summary.csv"  
 [33] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1428982-1448981-dynamic-1corestotal-allcorepera-2024041

In [7]:
summary_table

Parameter,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1008982-1028981-dynamic-1corestotal-allcorepera-20240416-154932-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1028982-1048981-dynamic-1corestotal-allcorepera-20240415-140831-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1048982-1068981-dynamic-1corestotal-allcorepera-20240415-141231-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1068982-1088981-dynamic-1corestotal-allcorepera-20240415-141631-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1088982-1108981-dynamic-1corestotal-allcorepera-20240415-142031-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-108982-128981-dynamic-1corestotal-allcorepera-20240416-154224-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1108982-1128981-dynamic-1corestotal-allcorepera-20240415-142431-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1128982-1148981-dynamic-1corestotal-allcorepera-20240415-142832-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1148982-1168981-dynamic-1corestotal-allcorepera-20240415-143232-summary.csv,⋯,libd_chr1-chr1_EA-libd_chr1-chr1_EA-868982-888981-dynamic-1corestotal-allcorepera-20240419-170914-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-888982-908981-dynamic-1corestotal-allcorepera-20240419-171216-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-88982-108981-dynamic-1corestotal-allcorepera-20240419-151129-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-8982-28981-dynamic-1corestotal-allcorepera-20240419-145925-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-8982-28981-dynamic-1corestotal-allcorepera-20240421-170133-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-908982-928981-dynamic-1corestotal-allcorepera-20240419-171517-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-928982-948981-dynamic-1corestotal-allcorepera-20240419-171818-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-948982-968981-dynamic-1corestotal-allcorepera-20240419-172119-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-968982-988981-dynamic-1corestotal-allcorepera-20240419-172420-summary.csv,libd_chr1-chr1_EA-libd_chr1-chr1_EA-988982-1008981-dynamic-1corestotal-allcorepera-20240419-172721-summary.csv
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
outdir,./output_chr1_a2_caud/,./output_chr1_a2/,./output_chr1_a2/,./output_chr1_a2/,./output_chr1_a2/,./output_chr1_a2_caud/,./output_chr1_a2/,./output_chr1_a2/,./output_chr1_a2/,⋯,./output_chr1_a2_caud/,./output_chr1_a2_caud/,./output_chr1_a2_caud/,./output_chr1_a2_caud/,./output_chr1_a2_caud/,./output_chr1_a2_caud/,./output_chr1_a2_caud/,./output_chr1_a2_caud/,./output_chr1_a2_caud/,./output_chr1_a2_caud/
chunk2,20000,20000,20000,20000,20000,20000,20000,20000,20000,⋯,20000,20000,20000,20000,20000,20000,20000,20000,20000,20000
methylation_data_path,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,⋯,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_EA.rda,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/

In [8]:
summary_table <- as.data.frame(t(summary_table))
colnames(summary_table) <- summary_table[1, ]
summary_table <- summary_table[-1, ]

In [9]:
#summary_table$number_cores <- as.numeric(as.character(summary_table$num_cores))
summary_table$number_cores <- 1

In [10]:
summary_table$runtime_seconds <- sapply(summary_table$runtime, function(x) {
  parts <- strsplit(x, ":")[[1]]
  as.numeric(parts[1]) * 3600 + as.numeric(parts[2]) * 60 + as.numeric(parts[3])
})

In [11]:
summary_table$runtime_hours <- summary_table$runtime_seconds/3600

In [15]:
min(summary_table$chunk2)

[1] "13721"

In [17]:
summary_table$chunk1 <- as.numeric(0)
summary_table$chunk2 <- as.numeric(as.character(summary_table$chunk2))

In [18]:
summary_table$runtime_core_hours <- summary_table$runtime_hours * summary_table$number_cores
summary_table$runtime_core_seconds <- summary_table$runtime_seconds * summary_table$number_cores
summary_table$core_hours_per_test <- summary_table$runtime_core_hours / (summary_table$chunk2 - summary_table$chunk1 + 1)
summary_table$core_seconds_per_test <- summary_table$runtime_core_seconds / (summary_table$chunk2 - summary_table$chunk1 + 1)

In [19]:
summary_table

,outdir,chunk2,methylation_data_path,tag,methInput_rds_path,cov,time_started,time_finished,runtime,type_CPU,⋯,physical_cores,scaffold_ID,number_cores,runtime_seconds,runtime_hours,runtime_core_hours,runtime_core_seconds,chunk1,core_hours_per_test,core_seconds_per_test
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
libd_chr1-chr1_AA-libd_chr1-chr1_AA-1008982-1028981-dynamic-1corestotal-allcorepera-20240416-154932-summary.csv,./output_chr1_a2_caud/,20000,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,libd_chr1-chr1_AA-1008982-1028981-dynamic-1corestotal-allcorepera-20240416-154932,/dcs04/lieber/statsgen/mnagle/mwas/pheno/caud/out/chr1_AA_1008982-1028981.rds,/dcs04/lieber/statsgen/mnagle/mwas/full_covariates/AA_caud.csv,2024-04-16 15:49:53,2024-04-17 14:41:20,22:51:27,AMD Opteron(TM) Processor 6234,⋯,24,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1008982-1028981-dynamic-1corestotal-allcorepera-20240416-154932,1,82287,22.85750000,22.85750000,82287,0,1.142818e-03,4.114144293
libd_chr1-chr1_AA-libd_chr1-chr1_AA-1028982-1048981-dynamic-1corestotal-allcorepera-20240415-140831-summary.csv,./output_chr1_a2/,20000,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,libd_chr1-chr1_AA-1028982-1048981-dynamic-1corestotal-allcorepera-20240415-140831,/dcs04/lieber/statsgen/mnagle/mwas/pheno/caud/out/chr1_AA_1028982-1048981.rds,/dcs04/lieber/statsgen/mnagle/mwas/full_covariates/AA_caud.csv,2024-04-15 14:08:43,2024-04-16 00:43:45,10:35:02,Intel(R) Xeon(R) CPU E5-2650L v3 @ 1.80GHz,⋯,24,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1028982-1048981-dynamic-1corestotal-allcorepera-20240415-140831,1,38102,10.58388889,10.58388889,38102,0,5.291680e-04,1.905004750
libd_chr1-chr1_AA-libd_chr1-chr1_AA-1048982-1068981-dynamic-1corestotal-allcorepera-20240415-141231-summary.csv,./output_chr1_a2/,20000,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,libd_chr1-chr1_AA-1048982-1068981-dynamic-1corestotal-allcorepera-20240415-141231,/dcs04/lieber/statsgen/mnagle/mwas/pheno/caud/out/chr1_AA_1048982-1068981.rds,/dcs04/lieber/statsgen/mnagle/mwas/full_covariates/AA_caud.csv,2024-04-15 14:12:44,2024-04-16 04:58:29,14:45:45,Intel(R) Xeon(R) CPU E5-2650L v3 @ 1.80GHz,⋯,24,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1048982-1068981-dynamic-1corestotal-allcorepera-20240415-141231,1,53145,14.76250000,14.76250000,53145,0,7.380881e-04,2.657117144
libd_chr1-chr1_AA-libd_chr1-chr1_AA-1068982-1088981-dynamic-1corestotal-allcorepera-20240415-141631-summary.csv,./output_chr1_a2/,20000,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,libd_chr1-chr1_AA-1068982-1088981-dynamic-1corestotal-allcorepera-20240415-141631,/dcs04/lieber/statsgen/mnagle/mwas/pheno/caud/out/chr1_AA_1068982-1088981.rds,/dcs04/lieber/statsgen/mnagle/mwas/full_covariates/AA_caud.csv,2024-04-15 14:16:45,2024-04-15 22:00:10,07:43:25,Intel(R) Xeon(R) CPU E5-2650 v4 @ 2.20GHz,⋯,24,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1068982-1088981-dynamic-1corestotal-allcorepera-20240415-141631,1,27805,7.72361111,7.72361111,27805,0,3.861612e-04,1.390180491
libd_chr1-chr1_AA-libd_chr1-chr1_AA-1088982-1108981-dynamic-1corestotal-allcorepera-20240415-142031-summary.csv,./output_chr1_a2/,20000,/dcs04/lieber/statsgen/shizhong/michael/mwas/pheno/caud/out/chr1_AA.rda,libd_chr1-chr1_AA-1088982-1108981-dynamic-1corestotal-allcorepera-20240415-142031,/dcs04/lieber/statsgen/mnagle/mwas/pheno/caud/out/chr1_AA_1088982-1108981.rds,/dcs04/lieber/statsgen/mnagle/mwas/full_covariates/AA_caud.csv,2024-04-15 14:21:17,2024-04-16 04:22:53,14:01:36,Intel(R) Xeon(R) CPU E5-2650L v3 @ 1.80GHz,⋯,24,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1088982-1108981-dynamic-1corestotal-allcorepera-20240415-142031,1,50496,14.02666667,14.02666667,50496,0,7.012983e-04,2.524673766
libd_chr1-chr1_AA-libd_chr1-chr1_AA-108982-128981-dynamic-1corestotal-allcorepera-20240416-154224-summary.csv,./output_chr1_a2_caud/,20000,/dcs04/lieber/

In [20]:
#summary_table[which.min(summary_table$core_hours_per_test), ]

In [21]:
data.table::fwrite(summary_table, "05-OUT_SLURM_benchmarks_a5_chr1_caud.csv")

## How long for whole dataset?

***This is for benchmarks with 500kb windows***

In [22]:
#summary_table <- summary_table[which(summary_table$num_cores == 1), ]

In [23]:
n_sites <- 26000000

In [25]:
#summary_table$core_hours_per_test

In [26]:
# Core hours for single run over methylation dataset (single brain region, single population dataset)

n_sites*mean(summary_table$core_hours_per_test)

[1] 16738.43

In [27]:
# Core hours for single run over methylation dataset (single brain region, single population dataset)

n_sites*mean(summary_table$core_hours_per_test)

[1] 16738.43

In [28]:
# Core hours for 3 x 3 brain regions x population datasets

brain_regions <- 3
populations <- 3
n_sites*mean(summary_table$core_hours_per_test)*brain_regions*populations

[1] 150645.9